# Completion profiling

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pyquickhelper.ipythonhelper import add_notebook_menu
add_notebook_menu()

## Function to profile

In [2]:
from mlstatpy.nlp.completion import CompletionTrieNode

def gain_dynamique_moyen_par_mot(queries, weights):
    per = list(zip(weights, queries))
    total = sum(weights) * 1.0
    res = []
    trie = CompletionTrieNode.build([(None, q) for _, q in per])
    trie.precompute_stat()
    trie.update_stat_dynamic()
    wks = [(w, p, len(w) - trie.min_keystroke0(w)[0]) for p, w in per]
    wks_dyn = [(w, p, len(w) - trie.min_dynamic_keystroke(w)[0])
               for p, w in per]
    wks_dyn2 = [(w, p, len(w) - trie.min_dynamic_keystroke2(w)[0])
                for p, w in per]
    gain = sum(g * p / total for w, p, g in wks)
    gain_dyn = sum(g * p / total for w, p, g in wks_dyn)
    gain_dyn2 = sum(g * p / total for w, p, g in wks_dyn2)
    ave_length = sum(len(w) * p / total for p, w in per)
    return gain, gain_dyn, gain_dyn2, ave_length

## Data

In [3]:
from mlstatpy.data.wikipedia import download_titles
file_titles = download_titles(country='fr')

In [4]:
from mlstatpy.data.wikipedia import enumerate_titles
list_titles = list(sorted(set(_ for _ in enumerate_titles(file_titles) if 'A' <= _[0] <= 'Z')))

In [5]:
import random
sample1000 = random.sample(list_titles, 1000)
with open("sample1000.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(sample1000))

## cProfile

In [29]:
import cProfile, io, pstats, os

def toprofile0(lines):
    gain_dynamique_moyen_par_mot(lines, [1.0] * len(lines))

def doprofile(lines, filename):
    pr = cProfile.Profile()
    pr.enable()
    toprofile0(lines)
    pr.disable()
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()
    rem = os.path.normpath(os.path.join(os.getcwd(), "..", "..", ".."))
    res = s.getvalue().replace(rem, "")
    ps.dump_stats(filename)
    return res

In [30]:
r = doprofile(sample1000, "completion.prof")
print(r)

         1478389 function calls in 1.832 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.832    1.832 <ipython-input-29-ff8ba7222a38>:3(toprofile0)
        1    0.001    0.001    1.831    1.831 <ipython-input-2-1cd63cc92e3c>:3(gain_dynamique_moyen_par_mot)
        1    0.156    0.156    1.220    1.220 \src\mlstatpy\nlp\completion.py:418(precompute_stat)
    15620    0.538    0.000    0.928    0.000 \src\mlstatpy\nlp\completion.py:489(merge_completions)
        1    0.117    0.117    0.499    0.499 \src\mlstatpy\nlp\completion.py:453(update_stat_dynamic)
    33238    0.175    0.000    0.295    0.000 \src\mlstatpy\nlp\completion.py:541(update_dynamic_minimum_keystroke)
    15620    0.213    0.000    0.217    0.000 {built-in method builtins.__build_class__}
    35224    0.093    0.000    0.101    0.000 \src\mlstatpy\nlp\completion.py:509(<listcomp>)
    33238    0.085    0.000    0.093    0.0

## memory_profiler

See [memory_profiler](https://pypi.python.org/pypi/memory_profiler/0.41).

In [8]:
from memory_profiler import profile
%load_ext memory_profiler

In [9]:
%memit toprofile(sample1000)

peak memory: 349.15 MiB, increment: 17.55 MiB


In [10]:
from io import StringIO
st = StringIO()
@profile(stream=st)
def toprofile(lines):
    gain_dynamique_moyen_par_mot(lines, [1.0] * len(lines))
toprofile(sample1000)

ERROR: Could not find file <ipython-input-10-7e3f7b9a5136>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.


In [11]:
%%file temp_mem_profile.py

from mlstatpy.nlp.completion import CompletionTrieNode
from memory_profiler import profile

@profile(precision=4)
def gain_dynamique_moyen_par_mot(queries, weights):
    per = list(zip(weights, queries))
    total = sum(weights) * 1.0
    res = []
    trie = CompletionTrieNode.build([(None, q) for _, q in per])
    trie.precompute_stat()
    trie.update_stat_dynamic()
    wks = [(w, p, len(w) - trie.min_keystroke0(w)[0]) for p, w in per]
    wks_dyn = [(w, p, len(w) - trie.min_dynamic_keystroke(w)[0])
               for p, w in per]
    wks_dyn2 = [(w, p, len(w) - trie.min_dynamic_keystroke2(w)[0])
                for p, w in per]
    gain = sum(g * p / total for w, p, g in wks)
    gain_dyn = sum(g * p / total for w, p, g in wks_dyn)
    gain_dyn2 = sum(g * p / total for w, p, g in wks_dyn2)
    ave_length = sum(len(w) * p / total for p, w in per)
    return gain, gain_dyn, gain_dyn2, ave_length

@profile(precision=4)
def toprofile():
    with open("sample1000.txt", "r", encoding="utf-8") as f:
        lines = [_.strip("\n\r ") for _ in f.readlines()]
    gain_dynamique_moyen_par_mot(lines, [1.0] * len(lines))
toprofile()

Overwriting temp_mem_profile.py


In [12]:
import sys
cmd = sys.executable
from pyquickhelper.loghelper import run_cmd
cmd += " -m memory_profiler temp_mem_profile.py"
out, err = run_cmd(cmd, wait=True)
print(out)

Filename: temp_mem_profile.py

Line #    Mem usage    Increment   Line Contents
     5  26.0273 MiB   0.0000 MiB   @profile(precision=4)
     6                             def gain_dynamique_moyen_par_mot(queries, weights):
     7  26.0508 MiB   0.0234 MiB       per = list(zip(weights, queries))
     8  26.0508 MiB   0.0000 MiB       total = sum(weights) * 1.0
     9  26.0508 MiB   0.0000 MiB       res = []
    10  33.4805 MiB   7.4297 MiB       trie = CompletionTrieNode.build([(None, q) for _, q in per])
    11  42.1641 MiB   8.6836 MiB       trie.precompute_stat()
    12  47.1914 MiB   5.0273 MiB       trie.update_stat_dynamic()
    13  47.2539 MiB   0.0625 MiB       wks = [(w, p, len(w) - trie.min_keystroke0(w)[0]) for p, w in per]
    14  47.3359 MiB   0.0820 MiB       wks_dyn = [(w, p, len(w) - trie.min_dynamic_keystroke(w)[0])
    15  47.3359 MiB   0.0000 MiB                  for p, w in per]
    16  47.4141 MiB   0.0781 MiB       wks_dyn2 = [(w, p, len(w) - trie.min_dynamic_keys

## pycallgraph

See [pycallgraph](http://pycallgraph.slowchop.com/en/develop/genindex.html), see also the [options](https://github.com/gak/pycallgraph/blob/develop/pycallgraph/output/graphviz.py) for Graphviz.

In [13]:
from pyquickhelper.helpgen.conf_path_tools import find_graphviz_dot
dot = find_graphviz_dot()

In [32]:
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

graphviz = GraphvizOutput()
graphviz.output_file = 'profile_dot.png'
graphviz.tool = dot
graphviz.font_size = 10

if False:
    # does not always work
    with PyCallGraph(output=graphviz):
        toprofile0(sample1000)        

In [20]:
from IPython.display import Image
Image(graphviz.output_file)

## pyinstrument

See [pyinstrument](https://github.com/joerick/pyinstrument).

In [35]:
from pyinstrument import Profiler

profiler = Profiler(use_signal=False)
profiler.start()

toprofile0(sample1000)

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

4.924 gain_dynamique_moyen_par_mot  <ipython-input-2-1cd63cc92e3c>:3
├─ 2.739 precompute_stat  mlstatpy\nlp\completion.py:418
│  ├─ 1.762 merge_completions  mlstatpy\nlp\completion.py:489
│  │  └─ 0.317 <listcomp>  mlstatpy\nlp\completion.py:509
│  ├─ 0.097 update_minimum_keystroke  mlstatpy\nlp\completion.py:528
│  ├─ 0.094 leaves  mlstatpy\nlp\completion.py:133
│  └─ 0.093 <genexpr>  mlstatpy\nlp\completion.py:438
├─ 1.925 update_stat_dynamic  mlstatpy\nlp\completion.py:453
│  ├─ 1.181 update_dynamic_minimum_keystroke  mlstatpy\nlp\completion.py:541
│  │  └─ 0.428 second_step  mlstatpy\nlp\completion.py:591
│  ├─ 0.125 unsorted_iter  mlstatpy\nlp\completion.py:98
│  └─ 0.069 init_dynamic_minimum_keystroke  mlstatpy\nlp\completion.py:612
└─ 0.175 build  mlstatpy\nlp\completion.py:205
   └─ 0.065 __init__  mlstatpy\nlp\completion.py:22



## SnakeViz

In [21]:
%load_ext snakeviz

L'instruction qui suit lance l'explorateur par défaut avec les données du profilage.

In [22]:
# %snakeviz toprofile(sample1000)

## line_profiler

See [line_profiler](https://github.com/rkern/line_profiler).

In [23]:
def lineprofile(lines):
    gain_dynamique_moyen_par_mot(lines, [1.0] * len(lines))

In [24]:
from mlstatpy.nlp.completion import CompletionTrieNode

In [25]:
from line_profiler import LineProfiler
prof = LineProfiler()
prof.add_function(gain_dynamique_moyen_par_mot)
prof.add_function(CompletionTrieNode.precompute_stat)
prof.run("lineprofile(sample1000)")
st = io.StringIO()
prof.print_stats(stream=st)
rem = os.path.normpath(os.path.join(os.getcwd(), "..", "..", ".."))
res = st.getvalue().replace(rem, "")
print(res)

Timer unit: 4.27635e-07 s

Total time: 4.03233 s
File: <ipython-input-2-1cd63cc92e3c>
Function: gain_dynamique_moyen_par_mot at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gain_dynamique_moyen_par_mot(queries, weights):
     4         1          708    708.0      0.0      per = list(zip(weights, queries))
     5         1           31     31.0      0.0      total = sum(weights) * 1.0
     6         1            6      6.0      0.0      res = []
     7         1       845977 845977.0      9.0      trie = CompletionTrieNode.build([(None, q) for _, q in per])
     8         1      5757749 5757749.0     61.1      trie.precompute_stat()
     9         1      2591918 2591918.0     27.5      trie.update_stat_dynamic()
    10         1        74224  74224.0      0.8      wks = [(w, p, len(w) - trie.min_keystroke0(w)[0]) for p, w in per]
    11         1           18     18.0      0.0      wks_dyn = [(w, p, len(w) -